<a href="https://colab.research.google.com/github/witold87/pub_haystack_unstructurized/blob/develop/unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install farm-haystack==0.8.0

You should consider upgrading via the '/home/wicio/PycharmProjects/pub_haystack/hay/bin/python -m pip install --upgrade pip' command.


In [2]:
import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.preprocessor.cleaning import clean_wiki_text


01/26/2022 22:10:40 - INFO - faiss.loader -   Loading faiss with AVX2 support.
01/26/2022 22:10:40 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
01/26/2022 22:10:40 - INFO - faiss.loader -   Loading faiss.
01/26/2022 22:10:40 - INFO - faiss.loader -   Successfully loaded faiss.
/home/wicio/PycharmProjects/pub_haystack/hay/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
01/26/2022 22:10:41 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", use_gpu=False)

01/26/2022 22:10:48 - INFO - farm.utils -   Using device: CPU 
01/26/2022 22:10:48 - INFO - farm.utils -   Number of GPUs: 0
01/26/2022 22:10:48 - INFO - farm.utils -   Distributed Training: False
01/26/2022 22:10:48 - INFO - farm.utils -   Automatic Mixed Precision: None


In [4]:
import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.preprocessor.cleaning import clean_wiki_text
#from memory_profiler import profile

def tutorial1_basic_qa_pipeline():
    logger = logging.getLogger(__name__)
    document_store = FAISSDocumentStore(faiss_index_factory_str='Flat')

    with open('nestle_report_de.txt', 'r') as file:
        text = file.readlines()

    text_str = '\n'.join([str(elem) for elem in text])
    stripped_text = text_str.strip('\n')
    print(stripped_text)

    dicts = [{'text': stripped_text,
              'meta': {'name': 'thai_air_th.txt'}}]
    document_store.write_documents(dicts)
    retriever = DensePassageRetriever(document_store=document_store)
    document_store.update_embeddings(retriever)

    from haystack.pipeline import ExtractiveQAPipeline

    pipe = ExtractiveQAPipeline(reader, retriever)


    # questions = ['Who is in the board of directors?',
    #              'What is the nature of business?',
    #              'When the company was started?',
    #              'Where the company operates?',
    #              'How many workers they have?']


    questions_en = ['What is the net debt?',
                    'Wie hoch ist der Umsatz in der Gruppe?',
                    'What is the revenue in group?',
                    'What is the net profit?']


    questions_th = ['หนี้สุทธิคืออะไร??',
                    'รายได้ในกลุ่มคืออะไร?',
                    'กำไรสุทธิคืออะไร?',
                    'how does the covid-19 affect the company?']
    
    questions_de = ['Wie hoch ist die Nettoverschuldung? ',
                    'Wie hoch ist der Umsatz in der Gruppe?' ]


    questions_fr = ['Quelle est la dette nette?',
                    'Quel est le revenu du groupe?',
                    'Quel est le chiffre daffaires du groupe? ']


    questions_es = ['¿Cuáles son el resultado de la crisis?',
                 '¿Cuál es la naturaleza del negocio?',
                 '¿Cuándo se fundó la empresa?',
                 ]

    def predict(questions, pipe):
      for question in questions:
          prediction = pipe.run(
              query=question
          )
          print(prediction['query'])
          print(prediction['answers'][0]['answer'])
          print(prediction['answers'][0]['probability'])
          print(prediction['answers'][0]['context'])

    predict(questions_en, pipe)



if __name__ == "__main__":
    tutorial1_basic_qa_pipeline()


Vorwort

Der Halbjahresbericht enthält bestimmte finanzielle

Leistungskennzahlen, die nicht nach IFRS definiert sind und

die von der Geschäftsleitung dazu verwendet werden, die fi­

nanzielle und die operative Leistung der Gruppe zu bewerten.

Dazu zählen unter anderem:

– das organische Wachstum, das interne Realwachstum

und Preisanpassungen;

– die zugrunde liegende operative Ergebnismarge und die

operative Ergebnismarge;

– die Nettoverschuldung;

– der freie Geldfluss; und

– der zugrunde liegende Gewinn je Aktie (Earnings per

Share EPS) und der EPS bei konstanten Wechselkursen.

Die Geschäftsleitung geht davon aus, dass diese nicht nach

IFRS definierten finanziellen Leistungskennzahlen hilfreiche

Informationen zur finanziellen und operativen Leistung der

Gruppe liefern.

Im Dokument «Alternative Leistungskennzahlen», das unter

www.nestle.com/investors/publications veröffentlicht ist,

werden diese nicht nach IFRS definierten finanziellen

Leistungskennzahlen definiert.

H

/usr/local/lib/python3.7/dist-packages/haystack/document_store/sql.py:294: SAWarning: relationship 'DocumentORM.Meta' will copy column document.id to column meta.document_id, which conflicts with relationship(s): 'DocumentORM.meta' (copies document.id to meta.document_id), 'MetaORM.Document' (copies document.id to meta.document_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="Document,meta"' to the 'DocumentORM.Meta' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  meta_orms = [MetaORM(name=key, value=value) for key, value in meta_fields.items()]
/usr/local/lib/python3.7/dist-packages/haystack/docume

What is the net debt?
CHF 38,5 Milliarden
0.9298259615898132
overschuldung

Die Nettoverschuldung stieg per 30. Juni 2021 auf

CHF 38,5 Milliarden, verglichen mit CHF 31,3 Milliarden am

31. Dezember 2020. Der A


Inferencing Samples: 100%|██████████| 1/1 [00:40<00:00, 40.27s/ Batches]


Wie hoch ist der Umsatz in der Gruppe?
CHF 1,4 Milliarden
0.1571597009897232
erzeichneten ein Wachstum

von 16,7% und der Umsatz kletterte auf CHF 1,4 Milliarden in

79 Märkten. Purina­Produkte für Heimtiere erzielten ein zwei­


Inferencing Samples: 100%|██████████| 1/1 [00:39<00:00, 39.78s/ Batches]


What is the revenue in group?
Umsatz der Gruppe

Das organische Wachstum erreichte 8,1%
0.03432765230536461


und den Geschäftsmix.

Resultate der Gruppe

Umsatz der Gruppe

Das organische Wachstum erreichte 8,1% und das interne

Realwachstum betrug 6,8%. Di


Inferencing Samples: 100%|██████████| 1/1 [00:40<00:00, 40.02s/ Batches]

What is the net profit?
CHF 7,0 Milliarden
0.35945063829421997
lage­

vermögen. Das operative Ergebnis erhöhte sich um 0,2% auf

CHF 7,0 Milliarden. Die zugrunde liegende operative Ergebnis­

marge belief sich auf


In [ ]:
!pip freeze | grep farm

farm==0.7.1
farm-haystack==0.8.0


# Nowa sekcja

total 56
drwxr-xr-x 4 root root 4096 Jan 19 10:44 mlruns
-rw-r--r-- 1 root root 9570 Jan 19 11:36 nestle_report_de.txt
-rw-r--r-- 1 root root 8625 Jan 19 11:36 nestle_report_en.txt
-rw-r--r-- 1 root root 9944 Jan 19 11:36 nestle_report_fr.txt
drwxr-xr-x 1 root root 4096 Jan  7 14:33 sample_data
-rw-r--r-- 1 root root 2831 Jan 19 10:01 spanish_report.txt
-rw-r--r-- 1 root root 1653 Jan 19 10:01 telefonica_report_en.txt
-rw-r--r-- 1 root root 1892 Jan 19 10:01 telefonica_report_es.txt
